# **검색량과 데이터 테이블 합치기**

In [61]:
import pandas as pd
import numpy as np

In [128]:
df = pd.read_csv("C:/Users/whileduck/Desktop/빈좌석/클래식_빈좌석포함데이터.csv",low_memory = False)
keyword = pd.read_csv("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/키워드사운드_예술의전당 콘서트홀_검색량.csv", encoding = 'euc-kr')

In [129]:
keyword.head()

,날짜,키워드,PC 검색량,모바일 검색량,총 검색량
0,2016-01-01,예술의전당 콘서트홀,10,30,40
1,2016-01-02,예술의전당 콘서트홀,30,40,70
2,2016-01-03,예술의전당 콘서트홀,30,30,60
3,2016-01-04,예술의전당 콘서트홀,70,40,110
4,2016-01-05,예술의전당 콘서트홀,60,40,100


##### **공연 최초 예매일 전 일주일 간의 누적 검색량을 검색량으로 생각하도록 하자**

In [130]:
pre_open_date = df.loc[~df['pre_open_date'].isna(),'pre_open_date'].unique().tolist()
open_date = df.loc[df['pre_open_date'].isna(), 'open_date'].unique().tolist()

total_open = pre_open_date + open_date

In [116]:
print('유니크한 전체 공연 시간 :',df['전체공연시간'].nunique())
print('유니크한 전체 오픈 시간 : ', len(total_open))

유니크한 전체 공연 시간 : 162
유니크한 전체 오픈 시간 :  150


##### **공연 별로 중복된 티켓 예매일이 존재한다**

In [117]:
display(df['pre_open_date'].value_counts() / 2505)
display(df['open_date'].value_counts() / 2505)

2021-07-02    3.0
2022-07-30    3.0
2021-09-19    2.0
2020-08-23    2.0
2021-09-11    2.0
             ... 
2019-12-20    1.0
2019-11-16    1.0
2019-11-29    1.0
2019-10-14    1.0
2023-03-25    1.0
Name: pre_open_date, Length: 78, dtype: float64

2021-07-03    3.0
2022-07-31    3.0
2021-09-12    3.0
2018-10-29    2.0
2022-06-26    2.0
             ... 
2019-09-28    1.0
2019-11-30    1.0
2019-08-03    1.0
2019-10-25    1.0
2023-03-26    1.0
Name: open_date, Length: 138, dtype: float64

# **각 공연 별로 첫 예매 시작 일주일 전의 누적 검색량을 데이터프레임에 병합하기**

In [158]:
def keyword_calculator(data, keyword_data , days):
    
    data = data.copy()
    data['누적검색량'] = 0 # 맨 처음 시리즈를 모두 0으로 초기화 
    
    unique_date_list = data['전체공연시간'].unique()
    
    for unq_date in unique_date_list:
        
        cond = data['전체공연시간'] == unq_date
        
        if np.sum(data.loc[cond,'pre_open_date'].isna()): # 만약 선예매가 열린 적 없다면
            
            base_days = data.loc[cond, 'open_date'].unique()[0]
        else: # 만약 선예매가 열린 적이 있다면 
            base_days = data.loc[cond, 'pre_open_date'].unique()[0]
            
        end_days = pd.to_datetime(base_days) # YYYY-MM-DD 형태가 검색의 end 부분 
            
        start_days = end_days - pd.Timedelta(days = days)
            
        keyword_cond = (keyword_data['날짜'] >= str(start_days)[:10]) & (keyword_data['날짜'] <= str(end_days)[:10])
        
        cumulative_searches = np.sum(keyword.loc[keyword_cond, '총 검색량'])
        
        data.loc[cond, '누적검색량'] = cumulative_searches
        
    return data
    

In [164]:
result = keyword_calculator(data = df,keyword_data = keyword, days = 7)

In [165]:
result.head()

,seat,층,블록,열,넘버,X,Y,Z,대칭점,무대_밑_좌우시야각,...,running_time,intermission,member_yn,공연연도,공연월,공연일,공연연월,전체공연시간,전체거래시간,누적검색량
0,1층 A블록1열 1,1층,A블록,1,1,1451,542,-93,1층 E블록1열 9,21,...,120,15,N,2018,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000
1,1층 A블록1열 2,1층,A블록,1,2,1406,555,-93,1층 E블록1열 8,22,...,120,15,N,2018,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000
2,1층 A블록1열 3,1층,A블록,1,3,1361,568,-93,1층 E블록1열 7,23,...,120,15,N,2018,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000
3,1층 A블록1열 4,1층,A블록,1,4,1315,580,-93,1층 E블록1열 6,23,...,120,15,N,2018,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000
4,1층 A블록1열 5,1층,A블록,1,5,1270,591,-93,1층 E블록1열 5,24,...,120,15,N,2018,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000


In [166]:
result['누적검색량'].describe()

count    405810.000000
mean        900.185185
std         386.835485
min         110.000000
25%         590.000000
50%         890.000000
75%        1170.000000
max        2010.000000
Name: 누적검색량, dtype: float64

In [167]:
file_path = 'C:/Users/whileduck/Desktop/빈좌석/'

result.to_csv(file_path + '클래식_빈좌석포함데이터.csv',index = False)